In [1]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
import pubmed as pm
import pandas as pd
import wget
import time
import os

In [2]:
path = f'{os.getcwd()}/PDBs'
url_download = 'https://dbaasp.org/exports/peptides.csv?id.value=&name.value=&sequence.value=&sequence.option=full&sequenceLength.value=&complexity.value=&synthesisType.value=&uniprot.value=&nTerminus.value=&cTerminus.value=&unusualAminoAcid.value=&intraChainBond.value=&interChainBond.value=&coordinationBond.value=&pubchem.value=&pubchemId.value=&threeDStructure.value=&kingdom.value=&source.value=&synergy.value=&articleAuthor.value=&articleJournal.value=&articleYear.value=&articleVolume.value=&articlePages.value=&articleTitle.value='

In [3]:
# Download Peptide List from DBBASP
wget.download(url_download)
os.system(f'mv peptides.csv {path}/DBBASP_releases/peptides.csv')

-1 / unknown

0

In [3]:
df = pd.read_csv(f'{path}/DBBASP_releases/peptides.csv')

In [4]:
#Add link from DBBASP for each peptide
base_url = 'https://dbaasp.org/peptide-card?id=DBAASP'
links = []
for i in range(0, len(df)):
    links.append(f'{base_url}{df.loc[i]["SYNTHESIS TYPE"][:1]}_{df.loc[i]["ID"]}')
df['LINK'] = links

In [5]:
driver = webdriver.Chrome()

In [7]:
# Get DOI from PubMed
doi_list = []

for link in df['LINK']:
    driver.get(link)
    time.sleep(2)
    try:
        table = driver.find_element(By.XPATH, '/html/body/main/div/main/div[1]/div[1]/div/div/div[18]/div/div[2]/div/div/div[2]/table/tbody')
        pubmed_links = table.find_elements(By.TAG_NAME, 'a')
        pmids = [pmid.get_attribute('href').split('pubmed/')[1] for pmid in pubmed_links]
        current_doi = []
        for pmid in pmids:
            current_doi.append(pm.get_doi_from_pmid(driver, pmid))
        doi_list.append(current_doi)
    except:
        doi_list.append('Not Found')
    
df['DOI'] = doi_list

In [11]:
driver.close()

In [9]:
dt = datetime.now()
df.to_csv(f'{path}/DBBASP_releases/DBBASP_{dt.strftime("%Y_%m_%d")}.csv', index=False)
df.to_csv(f'{path}/DBBASP.csv', index=False)

In [10]:
df

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT,LINK,DOI
0,1,Multimer,Distinctin,NaN,NLVSGLIEARKYLEQLHRKLKNCKV ENREVPPGFTALIKTLR...,NaN,Ribosomal,Gram+ Gram-,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPR_1,[https://doi.org/10.1016/s0014-5793(01)02324-9...
1,3,Multimer,Halocidin,NaN,WLNALLHHGLNCAKGVLA ALLHHGLNCAKGVLA,AMD AMD,Ribosomal,Gram+,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPR_3,[https://doi.org/10.1016/s0014-5793(02)02827-2...
2,4,Multimer,Khal,NaN,KWLNALLHHGLNCAKGVLA ALLHHGLNCAKGVLA,AMD AMD,Synthetic,Gram+ Gram-,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPS_4,"[https://doi.org/10.1128/AAC.01790-09, https:/..."
3,6,Multi-Peptide,Enterocin X,NaN,SNDSLWYGVGQFMGKQANCITNHPVKHMIIPGYCLSKILG IA...,NaN,Ribosomal,Gram+,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPR_6,[https://doi.org/10.1128/AEM.02264-09]
4,7,Multi-Peptide,EAFP1 + EAFP2,NaN,XTCASRCPRPCNAGLCCSIYGYCGSGNAYCGAGNCRCQCRG X...,NaN,Ribosomal,Fungus,Lipid Bilayer,https://dbaasp.org/peptide-card?id=DBAASPR_7,[https://doi.org/10.1016/s0014-5793(02)02829-6]
...,...,...,...,...,...,...,...,...,...,...,...
20415,20668,Monomer,Longicalycinin A linear analogue 12,NaN,FYPFI,NaN,Synthetic,NaN,NaN,https://dbaasp.org/peptide-card?id=DBAASPS_20668,[https://doi.org/10.1016/j.cbi.2019.108902]
20416,20669,Monomer,Longicalycinin A cyclic analogue 12,NaN,FYPFI,NaN,Synthetic,NaN,NaN,https://dbaasp.org/peptide-card?id=DBAASPS_20669,[https://doi.org/10.1016/j.cbi.2019.108902]
20417,20670,Monomer,Longicalycinin A linear analogue 13,NaN,VTPFL,NaN,Synthetic,NaN,NaN,https://dbaasp.org/peptide-card?id=DBAASPS_20670,[https://doi.org/10.1016/j.cbi.2019.108902]
20418,20671,Monomer,Longicalycinin A cyclic analogue 13,NaN,VTPFL,NaN,Synthetic,NaN,NaN,https://dbaasp.org/peptide-card?id=DBAASPS_20671,[https://doi.org/10.1016/j.cbi.2019.108902]
